<a href="https://colab.research.google.com/github/ZicoDiegoRR/stable_diffusion_xl_colab_ui/blob/main/stable_diffusion_xl_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###<font color="black"> » <b><font color="red">Installing Dependencies </b>💿</font> <font color="black"> «
#####ㅤRun this cell first before creating images!

In [ ]:
#@markdown Run this first to install essential libraries!
#@markdown Required to use the generator.
from IPython.display import clear_output
print("⚙️ | Downloading libraries...")
!git clone https://github.com/ZicoDiegoRR/stable_diffusion_xl_colab_ui.git StableDiffusionXLColabUI
!pip install -r StableDiffusionXLColabUI/requirements.txt
!pip install -r StableDiffusionXLColabUI/requirements_torch.txt
clear_output()
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
print("📁 | All essential libraries have been downloaded.")
print("🖌 | You can start generating images now.")

###<font color="black"> » <b><font color="orange">MultiControlNet<font color="black">, <b><font color="magenta"></b>IP-Adapter<font color="black">, and <b><font color="Lime">Inpainting</b> 🔧</font> <font color="black"> «

In [ ]:
from PIL import Image
from compel import Compel, ReturnedEmbeddingsType
from controlnet_aux import OpenposeDetector
from diffusers import ControlNetModel, StableDiffusionXLPipeline, StableDiffusionXLControlNetPipeline, AutoPipelineForInpainting, AutoencoderKL
from diffusers import DDPMScheduler, DPMSolverMultistepScheduler, DPMSolverSinglestepScheduler, KDPM2DiscreteScheduler, KDPM2AncestralDiscreteScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, HeunDiscreteScheduler, LMSDiscreteScheduler, DEISMultistepScheduler, UniPCMultistepScheduler, DDIMScheduler, PNDMScheduler
from diffusers.utils import load_image, make_image_grid
from huggingface_hub import login
from transformers import pipeline as pipe
from transformers import CLIPVisionModelWithProjection
from google.colab import drive
from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np
import time
import cv2
import re
import os
import subprocess
import os.path
import torch
import random
import json

#@markdown <b>Run the cell to start!</b>

#@markdown <small>Just run the cell and enjoy. (required to run the cell above first)</small>

#@markdown <small>You can disable Google Drive by not permitting the notebook to access your Google Drive storage.</small>

#@markdown <small>If the runtime got restarted, just run it again.</small>

# Function to load parameters config
def load_param(filename):
    try:
        with open(filename, 'r') as f:
            params = json.load(f)
            print(f"Found a config at {filename}.")
        return params
    except FileNotFoundError:
        return []

# Function to save the data to a json
def save_last(filename, data, type):
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                existing_data = json.load(file)
        else:
            existing_data = {}

        if type == "[Text-to-Image]":
            existing_data['text2img'] = data
        elif type == "[ControlNet]":
            existing_data['controlnet'] = data
        elif type == "[Inpainting]":
            existing_data['inpaint'] = data
        with open(filename, 'w') as file:
            json.dump(existing_data, file, indent=4)
    except Exception as e:
        print(f"Error occurred: {e}")

# Function to load last-generated image
def load_last(filename, type):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data.get(type, None)
    except (FileNotFoundError, json.JSONDecodeError):
        return None

# Function to load the saved data from a json
def load_number(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data['saved']
    except (FileNotFoundError, KeyError):
        return None

# Function to save the data to a json
def save_number(filename, data):
    with open(filename, 'w') as file:
        json.dump({'saved': data}, file)

#Function to save parameters config (had to make separate JSON def to avoid confusion)
def save_param(path, data):
    with open(path, 'w') as file:
        json.dump(data, file)

# Function to convert image into depth map
def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

# Only for display in output, nothing crazy
def get_depth_map_display(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    return image

# Function to restart the runtime to free up some of the VRAM if there's a change in model or the pipeline
def restart(new, old):
    print(f"New model is found. Your previous one ({old}) is different than your new one ({new}).")
    print("Restarting the runtime is necessary to load the new one.")
    time.sleep(2)
    print("Restarting the runtime...")
    time.sleep(0.5)
    os.kill(os.getpid(), 9)

# Loading the saved config for the IPyWidgets
try:
    drive.mount('/content/gdrive', force_remount=True)
except Exception as e:
    print("Excluding Google Drive storage...")
    time.sleep(1.5)
Save_and_Connect_To_GDrive = True if os.path.exists("/content/gdrive/MyDrive") else False
config_path_drive = os.path.join("/content/gdrive/MyDrive", "parameters.json")
config_path = os.path.join("/content", "parameters.json")
cfg_ver = load_param(config_path_drive)
cfg = load_param(config_path_drive) if cfg_ver else load_param(config_path)
if not cfg:
    print("No saved config found. Defaulting...")

# IPyWidgets⬇️
# ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

# Prompt Section
prompt_widget = widgets.Text(value=cfg[0] if cfg else "", description="Prompt", placeholder="Enter your prompt here")
model_widget = widgets.Text(value=cfg[1] if cfg else "", description="Model", placeholder="HF's repository or direct URL")
model_format_widget = widgets.Dropdown(
    options=["Pickle Tensor (.ckpt)", "Safe Tensor (.safetensors)"],
    value=cfg[2] if cfg else "Safe Tensor (.safetensors)",
    description="Model Format",
)
negative_prompt_widget = widgets.Text(value=cfg[3] if cfg else "", description="Negative Prompt", placeholder="What you don't want to see?")
token_widget = widgets.Text(description="CivitAI Token", placeholder="Avoid 401 error from CivitAI")
general_settings = widgets.VBox([widgets.HTML(value="<b>Image Generation Prompt🖌️</b>"),
    prompt_widget,
    model_widget,
    model_format_widget,
    negative_prompt_widget,
    token_widget,
    widgets.HTML(value="For safety reason, your token <b>won't be saved</b>.")
])

# Image Generation Settings
width_slider = widgets.IntSlider(min=512, max=1536, step=64, value=cfg[4] if cfg else 1024, description="Width")
height_slider = widgets.IntSlider(min=512, max=1536, step=64, value=cfg[5] if cfg else 1024, description="Height")
steps_slider = widgets.IntText(value=cfg[7] if cfg else 12, description="Steps")
scale_slider = widgets.FloatSlider(min=1, max=12, step=0.1, value=cfg[8] if cfg else 6, description="Scale")
vae_link_widget = widgets.Text(value=cfg[9] if cfg else "", description="VAE Link", placeholder="VAE model link")
vae_config = widgets.Text(value=cfg[36] if cfg else "", description="VAE Config Link", placeholder="VAE config link")
clip_skip_slider = widgets.IntSlider(min=0, max=12, step=1, value=cfg[10] if cfg else 2, description="Clip Skip")
image_settings = widgets.VBox([
    width_slider,
    height_slider,
    steps_slider,
    scale_slider,
    vae_link_widget,
    vae_config,
    clip_skip_slider
])

# Scheduler Section
scheduler_dropdown = widgets.Dropdown(
    options=[
        "Default (defaulting to the model)", "DPM++ 2M", "DPM++ 2M SDE",
        "DPM++ SDE", "DPM2", "DDPM",
        "DPM2 a", "DDIM", "PNDM", "Euler", "Euler a", "Heun", "LMS",
        "DEISMultistep", "UniPCMultistep"
    ],
    value=cfg[6] if cfg else "Default (defaulting to the model)",
    description="Scheduler",
)
karras_bool = widgets.Checkbox(value=cfg[32] if cfg else False, description="Enable Karras")
vpred_bool = widgets.Checkbox(value=cfg[33] if cfg else False, description="Enable V-prediction")
sgmuniform_bool = widgets.Checkbox(value=cfg[34] if cfg else False, description="Enable SGMUniform")
res_betas_zero_snr = widgets.Checkbox(value=cfg[35] if cfg else False, description="Rescale beta zero SNR")
scheduler_settings = widgets.VBox([
    scheduler_dropdown,
    karras_bool,
    vpred_bool,
    sgmuniform_bool,
    res_betas_zero_snr,
    widgets.HTML(value="Rescaling the betas to have zero terminal SNR helps to achieve vibrant color, but not necessary.")
])

# LoRA Section
lora_urls_widget = widgets.Text(
    value=cfg[11] if cfg else "",
    description="LoRA URLs",
    placeholder="Enter LoRA URLs separated by commas"
)
weight_scale_widget = widgets.Text(
    value=cfg[12] if cfg else "",
    description="Weights",
    placeholder="Enter weights separated by commas"
)
lora_settings = widgets.VBox([
    lora_urls_widget,
    weight_scale_widget,
])

# ControlNet Section
canny_min_slider = widgets.IntSlider(min=10, max=500, step=5, value=cfg[13] if cfg else 100, description="Min Threshold")
canny_max_slider = widgets.IntSlider(min=100, max=750, step=5, value=cfg[14] if cfg else 240, description="Max Threshold")
canny_link_widget = widgets.Text(value=cfg[15] if cfg else "", description="Canny Link", placeholder="Image link")
canny_toggle = widgets.Checkbox(value=cfg[16] if cfg else False, description="Enable Canny")
canny_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[17] if cfg else 0.7, description="Canny Strength")
canny_settings = widgets.VBox([canny_min_slider, canny_max_slider, canny_link_widget, canny_toggle, canny_strength_slider])

depth_map_link_widget = widgets.Text(value=cfg[18] if cfg else "", description="DepthMap Link", placeholder="Image link")
depth_map_toggle = widgets.Checkbox(value=cfg[19] if cfg else False, description="Enable Depth Map")
depth_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[20] if cfg else 0.7, description="Depth Strength")
depth_settings = widgets.VBox([depth_map_link_widget, depth_map_toggle, depth_strength_slider])

openpose_link_widget = widgets.Text(value=cfg[21] if cfg else "", description="OpenPose Link", placeholder="Image link")
openpose_toggle = widgets.Checkbox(value=cfg[22] if cfg else False, description="Enable OpenPose")
openpose_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[23] if cfg else 0.7, description="OpenPose Strength")
openpose_settings = widgets.VBox([openpose_link_widget, openpose_toggle, openpose_strength_slider])

controlnet_settings = widgets.Accordion([canny_settings, depth_settings, openpose_settings])
controlnet_settings.set_title(0, "Canny📝")
controlnet_settings.set_title(1, "Depth Map🏔️")
controlnet_settings.set_title(2, "Open Pose🕺🏻")

# Inpainting Section
inpainting_image_dropdown = widgets.Combobox(
    options=[
        "pre-generated text2image image",
        "pre-generated controlnet image",
        "previous inpainting image"
    ],
    value=cfg[24] if cfg else "pre-generated text2image image",
    description="Inpainting Image",
    ensure_option=False
)
mask_image_widget = widgets.Text(value=cfg[25] if cfg else "", description="Mask Image", placeholder="Image link")
inpainting_toggle = widgets.Checkbox(value=cfg[26] if cfg else False, description="Enable Inpainting")
inpainting_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[27] if cfg else 0.9, description="Inpainting Strength")
inpainting_settings = widgets.VBox([
    inpainting_image_dropdown,
    mask_image_widget,
    inpainting_toggle,
    inpainting_strength_slider
])

# IP-Adapter Section
ip_adapter_dropdown = widgets.Dropdown(
    options=[
        "ip-adapter-plus_sdxl_vit-h.bin",
        "ip-adapter-plus-face_sdxl_vit-h.bin",
        "ip-adapter_sdxl_vit-h.bin",
        "None"
    ],
    value=cfg[28] if cfg else "None",
    description="IP-Adapter",
)
ip_image_link_widget = widgets.Text(value=cfg[29] if cfg else "", description="IP Image Link", placeholder="Image links separated by commas")
ip_adapter_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[30] if cfg else 0.8, description="Adapter Strength")
ip_settings = widgets.VBox([
    ip_adapter_dropdown,
    ip_image_link_widget,
    ip_adapter_strength_slider
])

# Miscellaneous
submit_button_widget = widgets.Button(disabled=False, button_style='', description="Generate")
dont_spam = widgets.HTML(value="Please <b>don't spam</b> the generate button!")
keep_generating = widgets.HTML(value="You still can generate even though the cell is complete executing.")
submit_display = widgets.VBox([submit_button_widget, dont_spam, keep_generating])

freeze_widget = widgets.Checkbox(description="Use the same seed", value=cfg[31] if cfg else False)
misc_settings = widgets.VBox([freeze_widget])
loaded_model = widgets.Text(value="")
loaded_pipeline = widgets.Text(value="")

# Accordion, Tab, and  UI display grouping
advanced_settings_ui = widgets.Accordion([image_settings, scheduler_settings, lora_settings, controlnet_settings, inpainting_settings, ip_settings, misc_settings])
advanced_settings_titles = ["Image Settings⚙️", "Scheduler Settings🖼️⚙️", "LoRA Settings📁🖌️", "ControlNet Settings 🖼️🔧", "Inpainting Settings🖼️🖌️", "IP-Adapter Settings 🖼️📝", "Miscellaneous➡️💽"]
for i, title in enumerate(advanced_settings_titles):
    advanced_settings_ui.set_title(i, title)

ui = widgets.Tab()
ui.children = [general_settings, advanced_settings_ui]
ui.set_title(0, "General Settings")
ui.set_title(1, "Advanced Settings")
# ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

# The IPyWidgets handler

def widget():
    clear_output()
    submit_display.layout.display = "inline-block"
    display(ui, submit_display)

def submit(_):
    submit_display.layout.display = "none"
    submit_button()

# Main logic
def submit_button():
    torch.backends.cudnn.benchmark=True
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:16"
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    Freeze = freeze_widget.value

    # Handling Google Drive and seed
    folder = "/content/gdrive/MyDrive/"
    filename = os.path.join(folder, "random_number.json")
    saved_number = load_number(filename)
    if not Freeze:
        # Generate a new random number if Freeze is False
        random_number = random.randint(1, 1000000000)
        save_number(filename, random_number)
        saved_number = load_number(filename)
    else:
        # Use the saved number if Freeze is True
        if saved_number is not None:
            saved_number = saved_number
        else:
            print("No saved seed found. Generating new one...")
            random_number = random.randint(1, 1000000000)
            save_number(filename, random_number)
            saved_number = load_number(filename)

    # Handling user's input⬇️
    # ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
    Prompt = prompt_widget.value
    Model = model_widget.value
    Model_Format = model_format_widget.value
    Negative_Prompt = negative_prompt_widget.value

    Width = width_slider.value
    Height = height_slider.value
    Steps = steps_slider.value
    Scale = scale_slider.value
    VAE_Link = vae_link_widget.value
    VAE_Config = vae_config.value
    Clip_Skip = clip_skip_slider.value

    Scheduler = scheduler_dropdown.value
    Karras = karras_bool.value
    V_Prediction = vpred_bool.value
    SGMUniform = sgmuniform_bool.value
    Rescale_betas_to_zero_SNR = res_betas_zero_snr.value

    LoRA_URLs = lora_urls_widget.value
    Weight_Scale = weight_scale_widget.value
    Token = token_widget.value

    minimum_canny_threshold = canny_min_slider.value
    maximum_canny_threshold = canny_max_slider.value
    Canny_Link = canny_link_widget.value
    Canny = canny_toggle.value
    Canny_Strength = canny_strength_slider.value

    DepthMap_Link = depth_map_link_widget.value
    Depth_Map = depth_map_toggle.value
    Depth_Strength = depth_strength_slider.value

    OpenPose_Link = openpose_link_widget.value
    Open_Pose = openpose_toggle.value
    Open_Pose_Strength = openpose_strength_slider.value

    Inpainting_Image = inpainting_image_dropdown.value
    Mask_Image = mask_image_widget.value
    Inpainting = inpainting_toggle.value
    Inpainting_Strength = inpainting_strength_slider.value

    IP_Adapter = ip_adapter_dropdown.value
    IP_Image_Link = ip_image_link_widget.value
    IP_Adapter_Strength = ip_adapter_strength_slider.value
    # ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

    # Selecting image
    if Save_and_Connect_To_GDrive:
        base_path = "/content/gdrive/MyDrive"
    else:
        base_path = "/content"
    last_generation_loading = os.path.join(base_path, "last_generation.json")
    if Canny:
        if Canny_Link == "inpaint":
            Canny_link = load_last(last_generation_loading, 'inpaint')
        elif Canny_Link == "controlnet":
            Canny_link = load_last(last_generation_loading, 'controlnet')
        elif not Canny_Link:
            Canny_link = load_last(last_generation_loading, 'text2img')
        else:
            Canny_link = Canny_Link
        if not Canny_link and not os.path.exists(Canny_link):
            print("No generated image found. Defaulting to Text-to-Image...")
        else:
            pipeline_type = "controlnet"
    else:
        Canny_link = ""
    if Depth_Map:
        if DepthMap_Link == "inpaint":
            Depthmap_Link = load_last(last_generation_loading, 'inpaint')
        elif DepthMap_Link == "controlnet":
            Depthmap_Link = load_last(last_generation_loading, 'controlnet')
        elif not DepthMap_Link:
            Depthmap_Link = load_last(last_generation_loading, 'text2img')
        else:
            Depthmap_Link = DepthMap_Link
        if not Depthmap_Link and not os.path.exists(Depthmap_Link):
            print("No generated image found. Defaulting to Text-to-Image...")
        else:
            pipeline_type = "controlnet"
    else:
        Depthmap_Link = ""
    if Open_Pose:
        if OpenPose_Link == "inpaint":
            Openpose_Link = load_last(last_generation_loading, 'inpaint')
        elif OpenPose_Link == "controlnet":
            Openpose_Link = load_last(last_generation_loading, 'controlnet')
        elif not OpenPose_Link:
            Openpose_Link = load_last(last_generation_loading, 'text2img')
        else:
            Openpose_Link = OpenPose_Link
        if Openpose_Link is None and not os.path.exists(Openpose_Link):
            print("No generated image found. Defaulting to Text-to-Image...")
        else:
            pipeline_type = "controlnet"
    else:
        Openpose_Link = ""
    active_inpaint = False
    if Inpainting:
        if Canny or Depth_Map or Open_Pose:
            raise TypeError("You checked both ControlNet and Inpainting, which will cause incompatibility issues during your run. As of now, there's no alternative way to merge StableDiffusionXLControlNetPipeline and StableDiffusionXLInpaintingPipeline without causing any issues. Perhaps you want to use only one of them?")
        if not Mask_Image:
            raise ValueError("You checked Inpainting while you're leaving Mask_Image empty. Mask_Image is required for Inpainting!")
        if Inpainting_Image == "pre-generated text2image image":
            inpaint_img = load_last(last_generation_loading, 'text2img')
        elif Inpainting_Image == "pre-generated controlnet image":
            inpaint_img = load_last(last_generation_loading, 'controlnet')
        elif Inpainting_Image == "previous inpainting image":
            inpaint_img = load_last(last_generation_loading, 'inpaint')
        else:
            inpaint_image = Inpainting_Image
        if inpaint_img is not None and os.path.exists(inpaint_img):
            pipeline_type = "inpaint"
            inpaint_image = load_image(inpaint_img).resize((1024, 1024))
            mask_image = load_image(Mask_Image).resize((1024, 1024))
            active_inpaint = True
            display(make_image_grid([inpaint_image, mask_image], rows=1, cols=2))
        else:
            print("No generated image found. Defaulting to Text-to-Image...")
    if not IP_Image_Link and IP_Adapter != "None":
        raise ValueError(f"You selected {IP_Adapter}, but left the IP_Image_Link empty. Please change the IP_Adapter to None or add at least one image in IP_Image_Link!")
    if not Canny_link and not Depthmap_Link and not Openpose_Link and not active_inpaint:
        pipeline_type = "text2img"

    # Saving parameters config 1st phase
    params = [
        prompt_widget.value,
        model_widget.value,
        model_format_widget.value,
        negative_prompt_widget.value,
        width_slider.value,
        height_slider.value,
        scheduler_dropdown.value,
        steps_slider.value,
        scale_slider.value,
        vae_link_widget.value,
        clip_skip_slider.value,
        lora_urls_widget.value,
        weight_scale_widget.value,
        canny_min_slider.value,
        canny_max_slider.value,
        canny_link_widget.value,
        canny_toggle.value,
        canny_strength_slider.value,
        depth_map_link_widget.value,
        depth_map_toggle.value,
        depth_strength_slider.value,
        openpose_link_widget.value,
        openpose_toggle.value,
        openpose_strength_slider.value,
        inpainting_image_dropdown.value,
        mask_image_widget.value,
        inpainting_toggle.value,
        inpainting_strength_slider.value,
        ip_adapter_dropdown.value,
        ip_image_link_widget.value,
        ip_adapter_strength_slider.value,
        freeze_widget.value,
        karras_bool.value,
        vpred_bool.value,
        sgmuniform_bool.value,
        res_betas_zero_snr.value,
        vae_config.value
    ]
    save_param(f"{base_path}/parameters.json", params)

    # Checking if previous loaded model or pipeline is the same as the new one
    if loaded_model.value and loaded_model.value != model_widget.value:
        restart(model_widget.value, loaded_model.value)
    if loaded_pipeline.value and loaded_pipeline.value != pipeline_type:
        restart(pipeline_type, loaded_pipeline.value)

    # Logic to handle ControlNet and/or MultiControlNets
    controlnets = []
    images = []
    controlnets_scale = []
    if Canny and Canny_link is not None and os.path.exists(Canny_link):
        controlnets.append(ControlNetModel.from_pretrained("diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True, low_cpu_mem_usage=True).to("cuda"))
        print("🏞️ | Converting image with Canny Edge Detection...")
        c_img = load_image(Canny_link)
        image_canny = np.array(c_img)
        image_canny = cv2.Canny(image_canny, minimum_canny_threshold, maximum_canny_threshold)
        image_canny = image_canny[:, :, None]
        image_canny = np.concatenate([image_canny, image_canny, image_canny], axis=2)
        canny_image = Image.fromarray(image_canny)
        print("✅ | Canny Edge Detection is complete.")
        time.sleep(1)
        display(make_image_grid([c_img, canny_image.resize((1024, 1024))], rows=1, cols=2))
        images.append(canny_image.resize((1024, 1024)))
        controlnets_scale.append(Canny_Strength)
    if Depth_Map and Depthmap_Link is not None:
        controlnets.append(ControlNetModel.from_pretrained("diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True, low_cpu_mem_usage=True).to("cuda"))
        print("🏞️ | Converting image with Depth Map...")
        image_depth = load_image(Depthmap_Link).resize((1024, 1024))
        depth_estimator = pipe("depth-estimation")
        depth_map = get_depth_map(image_depth, depth_estimator).unsqueeze(0).half().to("cpu")
        images.append(depth_map)
        depth_map_display = Image.fromarray(get_depth_map_display(image_depth, depth_estimator))
        print("✅ | Depth Map is complete.")
        controlnets_scale.append(Depth_Strength)
        time.sleep(1)
        display(make_image_grid([image_depth, depth_map_display], rows=1, cols=2))
    if Open_Pose and Openpose_Link is not None:
        openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet").to("cpu")
        controlnets.append(ControlNetModel.from_pretrained("thibaud/controlnet-openpose-sdxl-1.0", torch_dtype=torch.float16, low_cpu_mem_usage=True).to("cuda"))
        print("🏞️ | Converting image with Open Pose...")
        image_openpose = load_image(Openpose_Link)
        openpose_image = openpose(image_openpose)
        images.append(openpose_image.resize((1024, 1024)))
        print("✅ | Open Pose is done.")
        controlnets_scale.append(Open_Pose_Strength)
        display(make_image_grid([image_openpose, openpose_image.resize((1024, 1024))], rows=1, cols=2))

    image_encoder = CLIPVisionModelWithProjection.from_pretrained(
        "h94/IP-Adapter",
        subfolder="models/image_encoder",
        torch_dtype=torch.float16,
    ) if IP_Adapter != "None" else None

    # Logic to handle VAE
    if VAE_Link and VAE_Config:
        if not os.path.exists("/content/VAE"):
            os.mkdir("VAE")
        vae_filename = VAE_Link.replace("/", "_").replace(".", "_") + ".safetensors"
        if VAE_Link.startswith("http"):
            if "civitai.com" in VAE_Link:
                if "?" in VAE_Link or "&" in VAE_Link:
                    vae_link = VAE_Link + "&token=" + Token
                else:
                    vae_link = VAE_Link + "token=" + Token
            else:
                vae_link = VAE_Link
            if not os.path.exists(f"/content/VAE/{vae_filename}"):
                !cd /content/VAE; wget -O "$vae_filename" "$vae_link"
                !cd /content/VAE; wget -N "$VAE_Config"
        vae_path = f"/content/VAE/{vae_filename}" if VAE_Link.startswith("http") else VAE_Link
        vae = AutoencoderKL.from_single_file(vae_path, config="/content/VAE/config.json", torch_dtype=torch.float16, local_files_only=True)
    elif VAE_Link and not VAE_Config:
        vae = None
        print("You inputted a VAE link, but not the config. Config is essential to load the model.")
        print("Skipping VAE...")

    # Logic to differentiate if the model is Hugging Face's repository
    global pipeline
    if Model.count("/") == 1:
        if not controlnets and not active_inpaint and (pipeline_type != "text2img" or not loaded_pipeline.value) and (not loaded_model.value or model_widget.value != loaded_model.value):
            if VAE_Link:
                pipeline = StableDiffusionXLPipeline.from_pretrained(Model, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLPipeline.from_pretrained(Model, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
        elif active_inpaint and not controlnets and (pipeline_type != "inpaint" or not loaded_pipeline.value) and (not loaded_model.value or model_widget.value != loaded_model.value):
            if VAE_Link:
                pipeline = AutoPipelineForInpainting.from_pretrained(Model, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = AutoPipelineForInpainting.from_pretrained(Model, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
        elif (pipeline_type != "controlnet" or not loaded_pipeline.value) and (not loaded_model.value or model_widget.value != loaded_model.value):
            if VAE_Link:
                pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(Model, image_encoder=image_encoder, controlnet=controlnets, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(Model, image_encoder=image_encoder, controlnet=controlnets, torch_dtype=torch.float16).to("cuda")
    else:
        if not os.path.exists("/content/Checkpoint"):
            os.mkdir("Checkpoint")
        if ".ckpt" in Model_Format:
            format = ".ckpt"
        elif ".safetensors" in Model_Format:
            format = ".safetensors"
        checkpoint_name = f"checkpoint_model{format}"
        if Token and "civitai.com" in Model:
            if "?" in Model or "&" in Model:
                checkpoint_link = f"{Model}&token={Token}"
            else:
                checkpoint_link = f"{Model}token={Token}"
        else:
            checkpoint_link = Model
        Model_folder = Model.replace("/", "_").replace(".", "_")
        Model_path = f"/content/Checkpoint/{Model_folder}/{checkpoint_name}"
        Model_path_folder = f"/content/Checkpoint/{Model_folder}"
        if not os.path.exists(Model_path):
            if not os.path.exists(Model_path_folder):
                os.mkdir(Model_path_folder)
            !cd "$Model_path_folder"; wget -O "$checkpoint_name" "$checkpoint_link"
        try:
            if not controlnets and not active_inpaint and (pipeline_type != "text2img" or not loaded_pipeline.value) and (not loaded_model.value or model_widget.value != loaded_model.value):
                if VAE_Link:
                    pipeline = StableDiffusionXLPipeline.from_single_file(Model_path, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
                else:
                    pipeline = StableDiffusionXLPipeline.from_single_file(Model_path, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
            elif active_inpaint and not controlnets and (pipeline_type != "inpaint" or not loaded_pipeline.value) and (not loaded_model.value or model_widget.value != loaded_model.value):
                if VAE_Link:
                    pipeline = AutoPipelineForInpainting.from_single_file(Model_path, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
                else:
                    pipeline = AutoPipelineForInpainting.from_single_file(Model_path, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
            elif (pipeline_type != "controlnet" or not loaded_pipeline.value) and (not loaded_model.value or model_widget.value != loaded_model.value):
                if VAE_Link:
                    pipeline = StableDiffusionXLControlNetPipeline.from_single_file(Model_path, image_encoder=image_encoder, controlnet=controlnets, vae=vae, torch_dtype=torch.float16).to("cuda")
                else:
                    pipeline = StableDiffusionXLControlNetPipeline.from_single_file(Model_path, image_encoder=image_encoder, controlnet=controlnets, torch_dtype=torch.float16).to("cuda")
        except (ValueError, OSError):
            pass
            os.remove(Model_path)
            if not Token and "civitai.com" in Model:
                Warning = "You inputted a CivitAI's link, but your token is empty. It's possible that you got unauthorized access during the download."
            else:
                Warning = "Did you input the correct link? Or did you use the correct format?"
            raise TypeError(f"The link ({Model}) contains unsupported file or the download was corrupted. {Warning}")

    if not loaded_model.value:
        loaded_model.value = model_widget.value
    if not loaded_pipeline.value:
        loaded_pipeline.value = pipeline_type

    # Seed, safety checker, and memory attention (Xformers)
    pipeline.enable_xformers_memory_efficient_attention()
    generator = torch.Generator("cpu").manual_seed(saved_number)
    pipeline.safety_checker = None

    # Handling schedulers
    Prediction_type = "v_prediction" if V_Prediction else "epsilon"
    scheduler_args = {"prediction_type": Prediction_type,
                           "use_karras_sigmas": Karras,
                           "rescale_betas_zero_snr": Rescale_betas_to_zero_SNR
                           }
    if SGMUniform:
      scheduler_args["timestep_spacing"] = "trailing"
    Scheduler_used = ["", f"{Scheduler} ", "", "", ""]
    Scheduler_used[0] = "V-Prediction " if Prediction_type == "v_prediction" else ""
    Scheduler_used[2] = "Karras " if Karras else ""
    Scheduler_used[3] = "SGMUniform " if SGMUniform else ""
    Scheduler_used[4] = "with zero SNR betas rescaling" if Rescale_betas_to_zero_SNR else ""
    if Scheduler == "DPM++ 2M":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DPM++ 2M SDE":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, algorithm_type="sde-dpmsolver++", **scheduler_args)
    elif Scheduler == "DPM++ SDE":
        pipeline.scheduler = DPMSolverSinglestepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DPM2":
        pipeline.scheduler = KDPM2DiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DPM2 a":
        pipeline.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DDPM":
        pipeline.scheduler = DDPMScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "Euler":
        pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "Euler a":
        pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "Heun":
        pipeline.scheduler = HeunDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "LMS":
        pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DEISMultistep":
        pipeline.scheduler = DEISMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "UniPCMultistep":
        pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DDIM":
        pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "PNDM":
        pipeline.scheduler = PNDMScheduler.from_config(pipeline.scheduler.config, **scheduler_args)

    # Prompt weighting using Compel
    compel = Compel(tokenizer=[pipeline.tokenizer, pipeline.tokenizer_2], text_encoder=[pipeline.text_encoder, pipeline.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True], truncate_long_prompts=False)
    conditioning, pooled = compel([Prompt, Negative_Prompt])

    # Logic to load LoRA(s)
    if LoRA_URLs:
        lora_list = []
        lora_path = []
        lora_links = [word for word in re.split(r"\s*,\s*", LoRA_URLs) if word]
        if not os.path.exists("/content/LoRAs"):
            os.mkdir("LoRAs")
        if not Weight_Scale:
            scales_string = ["1"] * len(lora_links)
        elif Weight_Scale and len(re.split(r",| ,", Weight_Scale)) < len(lora_links):
            scales_string = re.split(r",| ,", Weight_Scale)
            for j in range(len(lora_links) - len(scales_string)):
                scales_string.append("1")
        else:
            scales_string = re.split(r"\s*,\s*", Weight_Scale)
        scales = [float(num) for num in scales_string if num]

        for i, link in enumerate(lora_links, start=1):
            path_file = os.path.join("/content/LoRAs", link.replace("/", "_").replace(".", "_"))
            if not os.path.exists(path_file) and "http" in link:
                os.makedirs(path_file)
            lora_name = link.replace("/", "_").replace(".", "_")
            lora_file_name = f"lora_{lora_name}.safetensors"
            if "civitai.com" in link and Token:
                if "&" in link or "?" in link:
                    civit_link = f"{link}&token={Token}"
                else:
                    civit_link = f"{link}?token={Token}"
                if not os.path.isfile(os.path.join(path_file, lora_file_name)):
                    !cd "$path_file"; wget -O "$lora_file_name" "$civit_link"
                lora_list.append(lora_file_name)
                lora_path.append(path_file)
            elif not link.startswith("/content"):
                if not os.path.isfile(os.path.join(path_file, lora_file_name)):
                    !cd "$path_file"; wget -O "$lora_file_name" "$link"
                lora_list.append(lora_file_name)
                lora_path.append(path_file)
            else:
                if link.startswith("/content/gdrive/MyDrive"):
                    constructed_gdrive_link = link
                else:
                    constructed_gdrive_link = f"/content/gdrive/MyDrive/{link}"
                link_from_gdrive = constructed_gdrive_link.split("/")
                lora_path.append("/".join([word for word in link_from_gdrive if ".safetensors" not in word]))
                lora_list.append(link_from_gdrive[-1])
        lora_weights = [word for word in lora_list if word.endswith(".safetensors")]
        lora_names = [word.replace(".safetensors", "") for word in lora_weights]
        for p in range(len(lora_weights)):
            try:
                pipeline.load_lora_weights(f"{lora_path[p]}/{lora_weights[p]}", adapter_name=lora_names[p])
            except (ValueError):
                print(f"Skipping {lora_weights[p]}...")
        pipeline.set_adapters(lora_names, adapter_weights=scales)
        print("LoRAs:")
        for lora in lora_weights:
            print(lora)
    torch.cuda.empty_cache()

    # Logic to handle image(s) for IP-Adapter + display
    if IP_Adapter != "None":
        adapter_image = []
        simple_Url = [word for word in re.split(r"\s*,\s*", IP_Image_Link) if word]
        for link in simple_Url:
            adapter_image.append(load_image(link))
        adapter_display = [element for element in adapter_image]
        if len(adapter_image) % 3 == 0:
            row = len(adapter_image)/3
        else:
            row = int(len(adapter_image)/3) + 1
            for i in range(3*row - len(adapter_image)):
                adapter_display.append(load_image("https://huggingface.co/IDK-ab0ut/BFIDIW9W29NFJSKAOAOXDOKERJ29W/resolve/main/placeholder.png"))
        print("Image(s) for IP-Adapter:")
        display(make_image_grid([element.resize((1024, 1024)) for element in adapter_display], rows=row, cols=3))
        image_embeds = [adapter_image]
        pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name=IP_Adapter)
        pipeline.set_ip_adapter_scale(IP_Adapter_Strength)
    torch.cuda.empty_cache()

    # Generate
    if not controlnets and not active_inpaint: # For Text2Img
        image_save = "[Text-to-Image]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
            ).images[0]
            pipeline.unload_ip_adapter()
    elif active_inpaint and not controlnets: # For Inpainting
        image_save = "[Inpainting]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                image=inpaint_image,
                mask_image=mask_image,
                generator=generator,
                strength=Inpainting_Strength,
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
                image=inpaint_image,
                mask_image=mask_image,
                strength=Inpainting_Strength,
            ).images[0]
            pipeline.unload_ip_adapter()
    else: # For ControlNet
        image_save = "[ControlNet]"
        if Inpainting: # Deprecated. Will raise an error if both ControlNet and Inpainting collide.
            '''
            if IP_Adapter == "None":
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    clip_skip=Clip_Skip,
                    num_inference_steps=Steps,
                    generator=generator,
                    width=Width,
                    height=Height,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale,
                    guidance_scale=Scale,
                ).images[0]
            else:
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    num_inference_steps=Steps,
                    ip_adapter_image=image_embeds,
                    width=Width,
                    height=Height,
                    guidance_scale=Scale,
                    clip_skip=Clip_Skip,
                    generator=generator,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale,
                ).images[0]
            '''
        else:
            if IP_Adapter == "None":
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    clip_skip=Clip_Skip,
                    num_inference_steps=Steps,
                    generator=generator,
                    width=Width,
                    height=Height,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale,
                    guidance_scale=Scale,
                ).images[0]
            else:
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    num_inference_steps=Steps,
                    ip_adapter_image=image_embeds,
                    width=Width,
                    height=Height,
                    guidance_scale=Scale,
                    clip_skip=Clip_Skip,
                    generator=generator,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale
                ).images[0]

    # Saving the image
    current_time = time.localtime()
    formatted_time = time.strftime("[%H-%M-%S %B %d, %Y]", current_time)
    if Save_and_Connect_To_GDrive:
        if image_save == "[Text-to-Image]":
            image_save_path = "/content/gdrive/MyDrive/Text2Img"
        elif image_save == "[ControlNet]":
            image_save_path = "/content/gdrive/MyDrive/ControlNet"
        else:
            image_save_path = "/content/gdrive/MyDrive/Inpainting"
    else:
        if image_save == "[Text-to-Image]":
            image_save_path = "/content/Text2Img"
        elif image_save == "[ControlNet]":
            image_save_path = "/content/ControlNet"
        else:
            image_save_path = "/content/Inpainting"
    if not os.path.exists(image_save_path):
            os.makedirs(image_save_path)
    split_prompt = re.split("\s*,\s*", Prompt.replace("<", "").replace(">", "").replace(":", "_").replace(";", "_"))
    prompt_name = " ".join(split_prompt)
    generated_image_raw_filename = f"{image_save} {formatted_time} {prompt_name}"
    generated_image_filename = generated_image_raw_filename[:251] if len(generated_image_raw_filename) > 255 else generated_image_raw_filename
    generated_image_savefile = f"{image_save_path}/{generated_image_filename}.png"
    image.save(generated_image_savefile)
    widget()

    # Saving parameters config 2nd phase
    params = [
        prompt_widget.value,
        model_widget.value,
        model_format_widget.value,
        negative_prompt_widget.value,
        width_slider.value,
        height_slider.value,
        scheduler_dropdown.value,
        steps_slider.value,
        scale_slider.value,
        vae_link_widget.value,
        clip_skip_slider.value,
        lora_urls_widget.value,
        weight_scale_widget.value,
        canny_min_slider.value,
        canny_max_slider.value,
        canny_link_widget.value,
        canny_toggle.value,
        canny_strength_slider.value,
        depth_map_link_widget.value,
        depth_map_toggle.value,
        depth_strength_slider.value,
        openpose_link_widget.value,
        openpose_toggle.value,
        openpose_strength_slider.value,
        inpainting_image_dropdown.value,
        mask_image_widget.value,
        inpainting_toggle.value,
        inpainting_strength_slider.value,
        ip_adapter_dropdown.value,
        ip_image_link_widget.value,
        ip_adapter_strength_slider.value,
        freeze_widget.value,
        karras_bool.value,
        vpred_bool.value,
        sgmuniform_bool.value,
        res_betas_zero_snr.value,
        vae_config.value
    ]
    save_param(f"{base_path}/parameters.json", params)

    # Handling last generated image
    last_generation_json = os.path.join(base_path, "last_generation.json")
    save_last(last_generation_json, generated_image_savefile, image_save)

    # Displaying the image, seed, and scheduler
    display(image)
    print(f"Scheduler: {''.join(Scheduler_used)}")
    print(f"Seed: {saved_number}")
    print(f"Image is saved at {generated_image_savefile}.")
    torch.cuda.empty_cache()

submit_button_widget.on_click(submit)
widget()


###<font color="black"> » <b><font color="purple">Information </b>✏️📄</font> <font color="black"> «
#####ㅤ
<small>• Text2Img image is saved in Text2Img folder. </small>

<small>• ControlNet-generated image is saved in ControlNet folder. (requires **Canny**, **Depth Map**, and/or **Open Pose** to be checked, as well as the direct link to the reference image) </small>

<small>• Inpainting-generated image is saved in Inpainting folder. (requires **Inpainting** to be checked, as well as inputting the image and the mask image)</small>

<small> • You can't combine Inpainting and ControlNet.</small>

<small>• IP-Adapter doesn't change the image name.</small>

<small>• You can load LoRAs from your Google Drive by inputting their path. As of now, only LoRAs are supported. </small>

<small>• For ControlNet, leave the image link blank to use the last generated Text2Img image as the reference. Input "inpaint" to use the last generated Inpainting image. And lastly, input "controlnet" to use the last generated ControlNet image. (requires **Canny**, **Depth Map**, **Inpainting**, and/or **Open Pose** to be checked) </small>

***

###<font color="black"> » <b><font color="cyan">Guide </b>🚶🏻📋</font> <font color="black"> «
#####ㅤ

<small> **Prompt:** Basically, this one tells the AI what do you want to see in the image. Sometimes, you have to be strict with your words to align the image with your imagination.

<small> **Model (**checkpoint**):** A saved state during an intense training. This is required to generate the image. The type of model you inputted affects the overall style.

<small> **Model Format:** This is pretty self-explanatory. If you want to use .safetensors model, then set it to "Safe Tensors."

<small> **Steps:** It's simply how many iterations the AI will do in order to generate the image. More doesn't always better. You can look for references online.

<small> **Scale (**Guidance Scale**):** This affects how closely related the image with the prompt. High value can be precise, but low value can add extra uniqueness.

<small> **VAE:** Stands for Variational Autoencoder. It basically controls the color of your image.

<small> **Clip Skip:** Lets the AI skip set amount of layers during generation.

<small> **LoRA:** Stands for Low-Rank Adaptation. It holds weight to be "fed" to the AI. In simple terms, LoRA guides the AI to draw specific characters, style, poses, and so much more. You also need to specify the LoRA's scale, similar to **Scale**.

<small> **ControlNet:** Basically a strict instruction based on the inputted image to generate image closely related to the reference.

<small> **Inpainting:** Redrawing an image, but with certain parts of the image changed, just like editing with Photoshop, but AI does the job for you.

<small> **IP-Adapter:** Similar to LoRA, but only follows the inputted image(s). This is stricter than LoRA and sometimes lacks generalization.

<small> **Negative Prompt:** The reverse version of **Prompt**. Instead of telling the AI what do you want, this tells the AI about what do you want to be removed from the image.